In [1]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# create dataset for best picture

In [2]:
oscar_df = pd.read_csv("data/the_oscar_award.csv")
oscar_df = oscar_df[oscar_df['year_ceremony'] >= 1937]
oscar_df['category'] = oscar_df['category'].str.lower() 
bp_mask = oscar_df['category'].str.contains('best|outstanding (motion)? picture')

C:\Users\Nick\AppData\Local\Temp\ipykernel_11700\3112563163.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  bp_mask = oscar_df['category'].str.contains('best|outstanding (motion)? picture')


## Feature Engineering
### Add Nominations and win count

In [3]:
#create all masks
actor_mask = oscar_df['category'].str.contains('actor( in a leading role)?$')
actress_mask = oscar_df['category'].str.contains('actress( in a leading role)?$')
director_mask = oscar_df['category'].str.contains('directing')
writer_mask = oscar_df['category'].str.contains('writing.*screenplay')
sup_actor_mask = oscar_df['category'].str.contains('actor in a supporting role')
sup_actress_mask = oscar_df['category'].str.contains('actress in a supporting role')

mask_dct = {
    'actor': actor_mask,
    'actress': actress_mask,
    'director': director_mask,
    'writer': writer_mask,
    'sup_actor': sup_actor_mask,
    'sup_actress': sup_actress_mask,
    'best_picture': bp_mask,
}

for k,v in mask_dct.items():
    oscar_df['category'] = np.where(v,k,oscar_df['category'])



big5_mask = actor_mask | actress_mask | bp_mask | director_mask | writer_mask
atl_mask = big5_mask | sup_actor_mask | sup_actress_mask

C:\Users\Nick\AppData\Local\Temp\ipykernel_11700\2362575656.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  actor_mask = oscar_df['category'].str.contains('actor( in a leading role)?$')
C:\Users\Nick\AppData\Local\Temp\ipykernel_11700\2362575656.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  actress_mask = oscar_df['category'].str.contains('actress( in a leading role)?$')


In [4]:
noms = oscar_df.groupby(['year_film', 'film']).size().rename('noms')
wins = oscar_df[oscar_df['winner']==True & ~bp_mask].groupby(['year_film', 'film'])['category'].nunique().rename('wins')

big5_noms = oscar_df[big5_mask].groupby(['year_film', 'film'])['category'].nunique().rename('big5_noms')
big5_wins = oscar_df[~bp_mask & big5_mask & oscar_df['winner']==True].groupby(['year_film', 'film'])['category'].nunique().rename('big5_wins')

atl_noms = oscar_df[atl_mask].groupby(['year_film', 'film'])['category'].nunique().rename('atl_noms')
atl_wins = oscar_df[~bp_mask & atl_mask & oscar_df['winner']==True].groupby(['year_film', 'film'])['category'].nunique().rename('atl_wins')

for ser in (noms,wins,big5_noms,big5_wins,atl_noms,atl_wins):
    oscar_df = pd.merge(oscar_df,ser,left_on=['year_film', 'film'],right_index=True,how='left')

oscar_df[['wins','big5_wins','atl_wins']] = oscar_df[['wins','big5_wins','atl_wins']].fillna(0)


In [5]:
noms_df = (~oscar_df[atl_mask].astype({'winner':int}).pivot_table(index=['year_ceremony','film'],columns='category',values='winner').isna())
wins_df = (~oscar_df[atl_mask&(oscar_df['winner'])].astype({'winner':int}).pivot_table(index=['year_ceremony','film'],columns='category',values='wins').isna())

#remove best_film from both datasets
noms_df = noms_df.drop('best_picture',axis=1)
wins_df = wins_df.drop('best_picture',axis=1)

In [6]:
oscar_df = pd.merge(oscar_df,noms_df,left_on=['year_ceremony','film'],right_index=True,how='left',suffixes=('','_noms'))
oscar_df = pd.merge(oscar_df,wins_df,left_on=['year_ceremony','film'],right_index=True,how='left',suffixes=('_noms','_wins'))


In [7]:
oscar_df.isna().sum()
oscar_df[['atl_noms','big5_noms']] = oscar_df[['atl_noms','big5_noms']].fillna(0)
noms_wins_cols = oscar_df.columns[oscar_df.columns.str.contains('_wins|_noms$')]
oscar_df[noms_wins_cols] = oscar_df[noms_wins_cols].fillna(False)

## Filter Best Picture, remove data leakage, split data

In [8]:
model_df = oscar_df[oscar_df['category']=='best_picture'].set_index(['year_ceremony','film']).drop('category',axis=1)
model_df.drop(['year_film','ceremony','name'],axis=1,inplace=True)
model_df = model_df.astype(int)


#remove data leakage
model_df[model_df['winner']==1][['wins','big5_wins','atl_wins']] = model_df[model_df['winner']==1][['wins','big5_wins','atl_wins']]

C:\Users\Nick\AppData\Local\Temp\ipykernel_11700\4136675931.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df[model_df['winner']==1][['wins','big5_wins','atl_wins']] = model_df[model_df['winner']==1][['wins','big5_wins','atl_wins']]


In [9]:
test_mask = model_df.index.get_level_values(0) >= 2007

train_df = model_df[~test_mask]
test_df = model_df[test_mask]

# Train Model

In [10]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB
model = GaussianNB()
model.fit(train_df.drop('winner',axis=1),train_df['winner'])

test_df['predict'] = model.predict_proba(test_df.drop('winner',axis=1))[:,1]

<ipython-input-10-c5098452f353>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['predict'] = model.predict_proba(test_df.drop('winner',axis=1))[:,1]


In [11]:
predictions = test_df.groupby('year_ceremony')['predict'].idxmax()
actual = test_df.groupby('year_ceremony')['winner'].idxmax()

In [12]:
(predictions == actual).sum()/len(predictions)

0.6470588235294118

## Create model based on actors


In [36]:
model_df

winner  noms  wins  big5_noms  \
year_ceremony film                                                     
1942          Blossoms in the Dust          0     4     2          2   
              Citizen Kane                  0     9     2          4   
              Here Comes Mr. Jordan         0     7     3          4   
              Hold Back the Dawn            0     6     1          3   
              How Green Was My Valley       1    10     4          3   
...                                       ...   ...   ...        ...   
2023          The Fabelmans                 0     7     1          4   
              Tár                           0     6     1          4   
              Top Gun: Maverick             0     6     2          2   
              Triangle of Sadness           0     3     1          3   
              Women Talking                 0     2     2          2   

                                       big5_wins  atl_noms  atl_wins  \
year_ceremony film                                                     
1942          Blossoms in the Dust             0         2         0   
              Citizen Kane                     1         4         1   
              Here Comes Mr. Jordan            1         5         1   
              Hold Back the Dawn               0         3         0   
              How Green Was My Valley          1         5         2   
...                                          ...       ...       ...   
2023          The Fabelmans                    0         5         0   
              Tár                              0         4         0   
              Top Gun: Maverick                0         2         0   
              Triangle of Sadness              0         3         0   
              Women Talking                    1         2         1   

                                       actor_noms  actress_noms  \
year_ceremony film                                                
1942          Blossoms in the Dust              0             1   
              Citizen Kane                      1             0   
              Here Comes Mr. Jordan             1             0   
              Hold Back the Dawn                0             1   
              How Green Was My Valley           0             0   
...                                           ...           ...   
2023          The Fabelmans                     0             1   
              Tár                               0             1   
              Top Gun: Maverick                 0             0   
              Triangle of Sadness               0             0   
              Women Talking                     0             0   

                                       director_noms  sup_actor_noms  \
year_ceremony film                                                     
1942          Blossoms in the Dust                 0               0   
              Citizen Kane                         1               0   
              Here Comes Mr. Jordan                1               1   
              Hold Back the Dawn                   0               0   
              How Green Was My Valley              1               1   
...                                              ...             ...   
2023          The Fabelmans                        1               1   
              Tár                                  1               0   
              Top Gun: Maverick                    0               0   
              Triangle of Sadness                  1               0   
              Women Talking                        0               0   

                                       sup_actress_noms  writer_noms  \
year_ceremony film                                                     
1942          Blossoms in the Dust                    0            0   
              Citizen Kane                            0            1   
              Here Comes Mr. Jordan                   0            1

In [66]:
#load database of cast
cast_pkl = open("cast_info.pickle", "rb")
cast_info = pickle.load(cast_pkl)
cast_db = pd.DataFrame(cast_info).T

#database is indexed by tmdb id, so load tmdb to get mapping
tmdb_pkl = open("tmdb_info.pickle", "rb")
tmdb_info = pickle.load(tmdb_pkl)
tmdb_df = pd.DataFrame(tmdb_info).T

#create a df just mapping the cast
cast_df = pd.merge(tmdb['id'],cast_db,left_on='id',right_index=True,how='left')['cast']

#only keep ids and then explode with id representing a 
cast_df = cast_df[~cast_df.isna()].apply(lambda x: [i['id'] for i in x])

In [143]:
#create dataframe
t = cast_df[~cast_df.isna()].apply(lambda x: [i['id'] for i in x]).explode().to_frame()

#remove duplicates within a movie
t = t.reset_index().drop_duplicates(subset=['level_0','level_1','cast'])#.set_index(['level_0','level_1'])
#pivot list to dataframe where cast value is a column and 1 represents that cast member was in that movie
t = t.groupby(['level_0','level_1','cast']).size().unstack(fill_value=0)

In [147]:
t.sum(axis=1).tail(20)

level_0  level_1                                 
2022     Puss in Boots: The Last Wish                29
         RRR                                         30
         Stranger at the Gate                         8
         Tell It like a Woman                        22
         The Banshees of Inisherin                   17
         The Batman                                  82
         The Boy, the Mole, the Fox and the Horse     4
         The Elephant Whisperers                      2
         The Fabelmans                               63
         The Martha Mitchell Effect                  15
         The Quiet Girl                              21
         The Red Suitcase                             6
         The Sea Beast                               23
         The Whale                                    7
         To Leslie                                   30
         Top Gun: Maverick                           47
         Triangle of Sadness                         3

In [138]:
t.reset_index().pivot(columns='cast')

IndexError: index 342215199 is out of bounds for axis 0 with size 342207050

In [39]:
tmdb = pd.read_csv('data/tmdb.csv',index_col=0)

#create multiIndex based on nested list
new_index = pd.MultiIndex.from_tuples([tuple(x[::-1]) for x in tmdb.index.str.split('_')])
tmdb.index = new_index

